## Armen Khachatrian
### ML for Economics
### Classifications

In [1]:
import pandas as pd
import matplotlib

Answers **FOR *BOTH* SECTIONS**
1. What is the accuracy of a naïve model that would always guess ‘THEFT’ and what is the accuracy of your model?

I calculated accuracy in two ways: Logistic Regression and Random Forest.

The accuracy of a naïve model that would always guess ‘THEFT’ =  0.2230425789010179

Accuracy(LR) = 0.7567

Accuracy(RF) = 0.7639

2. Are there any other metrics that you have computed to assess the performance of your model? If yes, discuss their values.

I just calculated score.

3. What approach did you use and why?

I used Random Forest because this method is easy to use with binory classification. Also I used Logistic Regression  to compare accuracy. Logistic Regression and Random Forest are well-used for classification problems

4. How would you improve your model if you had another hour / another week at your disposal?

1) For Binary Classification:

I should use transform and aggregative primitives. Some of them I used: Minute, Hour, Day, Month. But, unfortunately, I could not do "time_since_previous" (transform) and "avg_time_between" (aggregative), which will be beneficial for sure. I faced troubles with using dfs method and I should spend more time on it as before I once used this method but I had more than one entity and I had target entity.

2) For Multiclass Classification:

I tried to measure all 32 types of crime. Of course, it is impossible to get high accuracy due to the fact that some of them happened rarely. I should do feature matrix and calculate the importance of them. Unfortunately, I had the lack of time to do it.

## Binary Classification

In [2]:
data = pd.read_csv("chicago_crimes_data_2010_2017.csv")
data.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Latitude,Longitude,Year,train_or_test
0,1027327,8899599,HV573490,2012-11-22 11:00:00,023XX N NORDICA AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT/GARAGE(NON.RESID.),False,False,2512,25.0,36.0,18.0,14,41.923153,-87.802559,2012,train
1,720647,8417624,HT650924,2011-12-28 21:17:00,002XX W 118TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,SIDEWALK,True,False,522,5.0,34.0,53.0,15,41.679787,-87.627957,2011,train
2,32037,7348669,HS150223,2010-02-03 21:40:00,026XX W 24TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,1034,10.0,28.0,30.0,08B,41.848314,-87.691627,2010,train
3,532182,8124889,HT358665,2011-06-21 13:10:00,017XX W PRYOR AVE,0320,ROBBERY,STRONGARM - NO WEAPON,CTA BUS,False,False,2212,22.0,19.0,75.0,03,41.692042,-87.665924,2011,train
4,2420587,11164022,JA530938,2017-11-30 11:45:00,031XX W DIVERSEY AVE,0325,ROBBERY,VEHICULAR HIJACKING,STREET,False,False,1411,14.0,35.0,21.0,03,41.932040,-87.705287,2017,test


In [3]:
k = 0
for i in range(len(data['train_or_test'])):
    if data['train_or_test'][i] == "train":
        k += 1
print(k)

381267


In [4]:
q = 0
for i in range(len(data['Primary Type'])):
    if data['Primary Type'][i] == "THEFT":
        q += 1
print(q)

108905


In [5]:
naive_acc = q/len(data['Primary Type'])
print("The accuracy of a naïve model that would always guess ‘THEFT’ = ", naive_acc)

The accuracy of a naïve model that would always guess ‘THEFT’ =  0.2230425789010179


In [6]:
data.dtypes

Unnamed: 0                int64
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
Latitude                float64
Longitude               float64
Year                      int64
train_or_test            object
dtype: object

In [7]:
data["Date"] = pd.to_datetime(data["Date"])
data["Minute"] = data.Date.dt.minute
data["Hour"] = data.Date.dt.hour
data["Day"] = data.Date.dt.day
data["Month"] = data.Date.dt.month

In [8]:
print("Date : ",data['Date'].unique())
print("Block : ",data['Block'].unique())
print("Description : ",data['Description'].unique())
print("Latitude : ",data['Latitude'].unique())
print("Longitude : ",data['Longitude'].unique())
print("Location Description: ",data['Location Description'].unique())
print("Beat : ",data['Beat'].unique())
print("District : ",data['District'].unique())
print("Ward : ",data['Ward'].unique())
print("Community Area : ",data['Community Area'].unique())
print("Year : ",data['Year'].unique())

Date :  ['2012-11-22T11:00:00.000000000' '2011-12-28T21:17:00.000000000'
 '2010-02-03T21:40:00.000000000' ... '2011-12-05T20:25:00.000000000'
 '2017-05-21T14:50:00.000000000' '2012-01-28T18:50:00.000000000']
Block :  ['023XX N NORDICA AVE' '002XX W 118TH ST' '026XX W 24TH ST' ...
 '040XX W SCHUBERT AVE' '067XX N LEHIGH AVE' '072XX W IBSEN ST']
Description :  ['TO VEHICLE' 'UNLAWFUL POSS OF HANDGUN' 'DOMESTIC BATTERY SIMPLE'
 'STRONGARM - NO WEAPON' 'VEHICULAR HIJACKING' 'OVER $500'
 'FORCIBLE ENTRY' 'AUTOMOBILE' 'TO PROPERTY' 'FROM BUILDING'
 'OTHER OFFENSE' '$500 AND UNDER' 'AGGRAVATED: HANDGUN' 'RETAIL THEFT'
 'AGG PO HANDS NO/MIN INJURY' 'SIMPLE' 'POSS: SYNTHETIC DRUGS'
 'MANU/DEL:CANNABIS OVER 10 GMS' 'SEX OFFENDER: FAIL TO REGISTER'
 'AGGRAVATED: OTHER DANG WEAPON' 'HARASSMENT BY ELECTRONIC MEANS'
 'ATTEMPT FORCIBLE ENTRY' 'PURSE-SNATCHING'
 'FINANCIAL IDENTITY THEFT OVER $ 300' 'OTHER VEHICLE OFFENSE' 'TO LAND'
 'THEFT/RECOVERY: AUTOMOBILE' 'AGGRAVATED' 'MANU/DELIVER: HEROIN (WHI

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['Description'] = le.fit_transform(data['Description'])
#data['Date'] = le.fit_transform(data['Date'])
#data['Location Description'] = le.fit_transform(data['Location Description'])
data['Block'] = le.fit_transform(data['Block'])
data['District'] = le.fit_transform(data['District'])
data['Ward'] = le.fit_transform(data['Ward'])
data['Community Area'] = le.fit_transform(data['Community Area'])
data['Latitude'] = le.fit_transform(data['Latitude'])
data['Longitude'] = le.fit_transform(data['Longitude'])

data.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Community Area,FBI Code,Latitude,Longitude,Year,train_or_test,Minute,Hour,Day,Month
0,1027327,8899599,HV573490,2012-11-22 11:00:00,9923,1320,CRIMINAL DAMAGE,291,PARKING LOT/GARAGE(NON.RESID.),False,...,18,14,164148,2843,2012,train,0,11,22,11
1,720647,8417624,HT650924,2011-12-28 21:17:00,1357,143A,WEAPONS VIOLATION,298,SIDEWALK,True,...,53,15,4305,167431,2011,train,17,21,28,12
2,32037,7348669,HS150223,2010-02-03 21:40:00,11223,0486,BATTERY,113,RESIDENCE,False,...,30,08B,103280,86478,2010,train,40,21,3,2
3,532182,8124889,HT358665,2011-06-21 13:10:00,7807,0320,ROBBERY,273,CTA BUS,False,...,75,03,8663,118741,2011,train,10,13,21,6
4,2420587,11164022,JA530938,2017-11-30 11:45:00,13207,0325,ROBBERY,312,STREET,False,...,21,03,171073,70150,2017,test,45,11,30,11


In [10]:
d1 = data.loc[data['Primary Type'] == "THEFT"]
d1["Binary Type"] = 1

/Users/a.khachatryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
d2 = data.loc[data['Primary Type'] != "THEFT"]
d2["Binary Type"] = 0

/Users/a.khachatryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
data = pd.concat([d1, d2], axis=0)

In [13]:
data_train = data.loc[data['train_or_test']== 'train'][:]
data_train

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,FBI Code,Latitude,Longitude,Year,train_or_test,Minute,Hour,Day,Month,Binary Type
5,1245448,9301182,HW445774,2013-08-05 12:00:00,15504,0810,THEFT,212,OTHER,False,...,06,185485,135592,2013,train,0,12,5,8,1
9,90734,7439846,HS242094,2010-04-04 20:45:00,25197,0890,THEFT,138,NURSING HOME/RETIREMENT HOME,False,...,06,210555,105071,2010,train,45,20,4,4,1
12,1079376,8983821,HW130715,2013-01-24 20:40:00,20368,0820,THEFT,0,STREET,True,...,06,200538,4475,2013,train,40,20,24,1,1
15,1361980,9437742,HW581784,2013-12-23 13:40:00,20538,0860,THEFT,253,SMALL RETAIL STORE,True,...,06,79827,164728,2013,train,40,13,23,12,1
16,1493294,9663883,HX314081,2014-06-22 12:00:00,21871,0820,THEFT,0,RESIDENCE,False,...,06,202991,53026,2014,train,0,12,22,6,1
18,1780637,10155198,HY342784,2015-07-14 19:30:00,1624,0810,THEFT,212,CTA PLATFORM,False,...,06,131392,56482,2015,train,30,19,14,7,1
19,1097298,9011425,HW158923,2013-02-16 11:56:00,27541,0820,THEFT,0,VACANT LOT/LAND,True,...,06,22250,157521,2013,train,56,11,16,2,1
22,1827983,10232733,HY420721,2015-09-12 17:15:00,233,0860,THEFT,253,DEPARTMENT STORE,False,...,06,127202,167539,2015,train,15,17,12,9,1
27,1343513,9407795,HW551119,2013-11-28 11:00:00,29582,0860,THEFT,253,GROCERY FOOD STORE,True,...,06,4425,124111,2013,train,0,11,28,11,1
31,250106,7684536,HS490690,2010-08-30 00:00:00,1324,0810,THEFT,212,SIDEWALK,False,...,06,122946,157870,2010,train,0,0,30,8,1


In [14]:
data_test = data.loc[data['train_or_test']== 'test'][:]
data_test

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,FBI Code,Latitude,Longitude,Year,train_or_test,Minute,Hour,Day,Month,Binary Type
50,1966665,10470274,HZ209113,2016-03-30 06:00:00,10739,0820,THEFT,0,RESIDENTIAL YARD (FRONT/BACK),False,...,06,101933,73446,2016,test,0,6,30,3,1
63,2042038,10590880,HZ341922,2016-07-07 15:30:00,16954,0820,THEFT,0,RESIDENCE,False,...,06,190275,115810,2016,test,30,15,7,7,1
75,2301153,10994261,JA321566,2017-06-25 03:15:00,23335,0860,THEFT,253,SMALL RETAIL STORE,False,...,06,206453,127772,2017,test,15,3,25,6,1
80,2393316,11126753,JA482123,2017-10-22 16:00:00,2944,0820,THEFT,0,STREET,False,...,06,137464,140208,2017,test,0,16,22,10,1
84,2033844,10577075,HZ325776,2016-06-27 15:15:00,22740,0820,THEFT,0,DEPARTMENT STORE,True,...,06,204955,58333,2016,test,15,15,27,6,1
96,2404768,11142702,JA502564,2017-11-07 13:00:00,12228,0820,THEFT,0,APARTMENT,False,...,06,173330,86131,2017,test,0,13,7,11,1
97,2337454,11050311,JA386075,2017-08-09 14:00:00,22150,0820,THEFT,0,GROCERY FOOD STORE,False,...,06,203335,26692,2017,test,0,14,9,8,1
113,2374055,11100243,JA447407,2017-09-26 11:52:00,9021,0860,THEFT,253,SMALL RETAIL STORE,False,...,06,53387,201084,2017,test,52,11,26,9,1
116,1952676,10443710,HZ181894,2016-03-10 12:50:00,849,0890,THEFT,138,RESTAURANT,False,...,06,123900,165761,2016,test,50,12,10,3,1
163,2008620,10537695,HZ281897,2016-05-26 22:30:00,16252,0860,THEFT,253,DRUG STORE,False,...,06,101576,45276,2016,test,30,22,26,5,1


In [15]:
del data_train['Date']
del data_train['Unnamed: 0']
del data_train['ID']
del data_train['Case Number']
del data_train['IUCR']
del data_train['Primary Type']
del data_train['Description']
del data_train['Arrest']
del data_train['Domestic']
del data_train['FBI Code']
del data_train['train_or_test']
del data_train['Location Description']

del data_test['Date']
del data_test['Unnamed: 0']
del data_test['ID']
del data_test['Case Number']
del data_test['IUCR']
del data_test['Primary Type']
del data_test['Description']
del data_test['Arrest']
del data_test['Domestic']
del data_test['FBI Code']
del data_test['train_or_test']
del data_test['Location Description']

In [16]:
from sklearn.ensemble import RandomForestClassifier

y_train = data_train["Binary Type"]
X_train = data_train[["Block", "Beat", "District", "Ward", "Community Area", "Latitude", "Longitude", 
                     "Year", "Minute", "Hour", "Day", "Month"]]
y_test = data_test["Binary Type"]
X_test = data_test[["Block", "Beat", "District", "Ward", "Community Area", "Latitude", "Longitude", 
                     "Year", "Minute", "Hour", "Day", "Month"]]

In [17]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0).fit(X_train, y_train)
RF.predict(X_test)
round(RF.score(X_test, y_test), 4)

0.7639

In [18]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
LR.predict(X_test)
round(LR.score(X_test,y_test), 4)

0.7567

## Multiclass classification

In [19]:
data['Primary Type'] = le.fit_transform(data['Primary Type'])
del data['Binary Type']

In [20]:
data_train = data.loc[data['train_or_test']== 'train'][:]
data_test = data.loc[data['train_or_test']== 'test'][:]

In [21]:
del data_train['Date']
del data_train['Unnamed: 0']
del data_train['ID']
del data_train['Case Number']
del data_train['IUCR']
del data_train['Description']
del data_train['Arrest']
del data_train['Domestic']
del data_train['FBI Code']
del data_train['train_or_test']
del data_train['Location Description']

del data_test['Date']
del data_test['Unnamed: 0']
del data_test['ID']
del data_test['Case Number']
del data_test['IUCR']
del data_test['Description']
del data_test['Arrest']
del data_test['Domestic']
del data_test['FBI Code']
del data_test['train_or_test']
del data_test['Location Description']

In [22]:
y_train = data_train["Primary Type"]
X_train = data_train[["Block", "Beat", "District", "Ward", "Community Area", "Latitude", "Longitude", 
                     "Year", "Minute", "Hour", "Day", "Month"]]
y_test = data_test["Primary Type"]
X_test = data_test[["Block", "Beat", "District", "Ward", "Community Area", "Latitude", "Longitude", 
                     "Year", "Minute", "Hour", "Day", "Month"]]

In [23]:
#import numpy as np
#y_train = np.log(y_train + 1) 
#y_test = np.log(y_test + 1)

In [25]:
import numpy as np
random_state = np.random.RandomState(0)
LR = LogisticRegression(random_state=random_state, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
LR.predict(X_test)
round(LR.score(X_test,y_test), 4)

/Users/a.khachatryan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.2518

In [26]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0).fit(X_train, y_train)
RF.predict(X_test)
round(RF.score(X_test, y_test), 4)

0.2863

In [27]:
LR1 = LogisticRegression(random_state=random_state, solver='sag', multi_class='multinomial').fit(X_train, y_train)
LR1.predict(X_test)
round(LR1.score(X_test,y_test), 4)

/Users/a.khachatryan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.2557

In [ ]:
#IMPORTANCE
#feature_importances(model, feature_matrix.columns, n=25)